In [34]:
import os, requests
import pandas as pd
import nglview as nv
import matplotlib.pyplot as plt
from ipywidgets import Output, HBox

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **MD_id**: PDB code of the protein structure (e.g. [A01M6](https://mmb-dev.mddbr.eu/#/id/A01M6/overview))

In [33]:
MD_id = 'A01M6'
steps = 1500
lipid_sel = '(resname DPPC and name P8)'
ngl_sel = 'DPPC'
ngl_hd_sel = f'{ngl_sel} and _P'

# common arguments for all the components
input_top_path = f'data/{MD_id}.pdb'
input_traj_path = f'data/{MD_id}.xtc'

<a id="fetch"></a>
***
## Fetching structure and trajectory

In [8]:
# Check if folders 'out' and 'tmp' exist and create them if not
for folder in ['out', 'data']:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Created folder: {folder}")

# Check if PDB and XTC files already exist
if not os.path.exists(f"data/{MD_id}.pdb"):
    # Download PDB
    headers = {'accept': 'chemical/x-pdb'}
    url = f"https://mmb-dev.mddbr.eu/api/rest/v1/projects/{MD_id}/files/structure.pdb"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        pdb_content = response.text
        # Save the PDB content to a file
        with open(f"{MD_id}.pdb", 'w') as pdb_file:   
            pdb_file.write(pdb_content)
        print(f"Saved PDB file")
    else:
        print(f"Error for the PDB file: {response.status_code}")

if not os.path.exists(f"data/{MD_id}.xtc"): 
    # Download XTC
    headers2 = {'accept': 'application/octet-stream'}
    url2= f'https://mmb-dev.mddbr.eu/api/rest/v1/projects/{MD_id}/files/trajectory.xtc'
    response2 = requests.get(url2, headers=headers2)
    if response2.status_code == 200:
        xtc_content = response2.content
        with open(f"{MD_id}.xtc", 'wb') as xtc_file:
            xtc_file.write(xtc_content)
        print(f"Saved XTC file")
    else:
        print(f"Error for the XTC file: {response2.status_code}")

In [31]:
view = nv.show_file(input_top_path)
view

NGLWidget()

<a id="membrane"></a>
***
## Membrane identification

Membrane identification...<br>
***
**Building Blocks** used:
 - [FatslimMembranes](https://biobb-mem.readthedocs.io/en/latest/fatslim.html#module-fatslim.fatslim_membranes) from **biobb_mem.fatslim.fatslim_membranes**
***

In [12]:
from biobb_mem.fatslim.fatslim_membranes import fatslim_membranes

prop = {
    'selection': lipid_sel,
    'cutoff': 2.2
}
fatslim_membranes(input_top_path=input_top_path,
                  input_traj_path=input_traj_path,
                  output_ndx_path="out/leaflets.ndx",
                  properties=prop)

2024-12-05 15:12:49,614 [MainThread  ] [INFO ]  Module: biobb_mem.fatslim.fatslim_membranes Version: 5.0.2
2024-12-05 15:12:49,615 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_3cafefda-947b-4ba4-b79e-9ffc345dc5b6 directory successfully created
2024-12-05 15:12:49,627 [MainThread  ] [INFO ]  Copy: data/A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_3cafefda-947b-4ba4-b79e-9ffc345dc5b6
2024-12-05 15:12:50,640 [MainThread  ] [INFO ]  Copy: data/A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_3cafefda-947b-4ba4-b79e-9ffc345dc5b6
2024-12-05 15:12:50,988 [MainThread  ] [INFO ]  gmx editconf -f /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_3cafefda-947b-4ba4-b79e-9ffc345dc5b6/A01M6.pdb -o 4663c9ca-2fc9-494c-a12e-4a5322babf42/output.gro -box 147.3093 147.3093 177.3872 ; fatslim membranes -n 7e2dc93c-702b-42a2-b0de-f0e92d367ea3/headgroups.ndx -c 4663c9ca-2fc9-494c-a12e-4a5322babf4

0

In [13]:
from biobb_mem.fatslim.fatslim_membranes import parse_index
parse_index??

Signature: parse_index(ndx)
Source:   
def parse_index(ndx):
    """
    Parses a GROMACS index file (.ndx) to extract leaflet groups.

    Args:
        ndx (str): Path to the GROMACS index file (.ndx).
    Returns:
        dict: A dictionary where keys are group names and values are lists of integers representing atom indices.
    """

    # Read the leaflet.ndx file
    with open(ndx, 'r') as file:
        leaflet_data = file.readlines()

    # Initialize dictionaries to store leaflet groups
    leaflet_groups = {}
    current_group = None

    # Parse the leaflet.ndx file
    for line in leaflet_data:
        line = line.strip()
        if line.startswith('[') and line.endswith(']'):
            current_group = line[1:-1].strip()
            leaflet_groups[current_group] = []
        elif current_group is not None:
            leaflet_groups[current_group].extend(map(int, line.split()))
    return leaflet_groups
File:      ~/repo/biobb_mem/biobb_mem/fatslim/fatslim_membranes.py
Typ

In [15]:
from biobb_mem.fatslim.fatslim_membranes import display_fatslim
view = display_fatslim(input_top_path,output_ndx_path="out/leaflets.ndx")
view.add_ball_and_stick(selection=ngl_hd_sel, color='white',aspect_ratio=5) # headgroups
view

NGLWidget()

<a id="density"></a>
***
## Density profile

Calculate density... [amber selection masks](https://amberhub.chpc.utah.edu/atom-mask-selection-syntax/)<br>
***
**Building Blocks** used:
 - [CpptrajDensity](https://biobb-mem.readthedocs.io/en/latest/ambertools.html#module-ambertools.cpptraj_density) from **biobb_mem.ambertools.cpptraj_density**
***

In [35]:
from biobb_mem.ambertools.cpptraj_density import cpptraj_density

prop = {
    'mask': '::M ::A,B',
    'steps': steps,
}
cpptraj_density(input_top_path=input_top_path,
            input_traj_path=input_traj_path,
            output_cpptraj_path='out/density_mask.dat',
            properties=prop)

2024-12-05 15:24:18,789 [MainThread  ] [INFO ]  Module: biobb_mem.ambertools.cpptraj_density Version: 5.0.2
2024-12-05 15:24:18,790 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_e130a480-f3dd-4c6e-a1d9-3e64bff3c1bb directory successfully created
2024-12-05 15:24:18,792 [MainThread  ] [INFO ]  Copy: data/A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_e130a480-f3dd-4c6e-a1d9-3e64bff3c1bb
2024-12-05 15:24:19,367 [MainThread  ] [INFO ]  Copy: data/A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_e130a480-f3dd-4c6e-a1d9-3e64bff3c1bb
2024-12-05 15:24:19,367 [MainThread  ] [INFO ]  cpptraj -i 12090ea6-2d44-486b-8d7f-443f222c576c/instructions.in

2024-12-05 15:24:19,546 [MainThread  ] [INFO ]  Executing: cpptraj -i 12090ea6-2d44-486b-8d7f-443f222c576c/instructions.in...
2024-12-05 15:24:19,547 [MainThread  ] [INFO ]  Exit code: 0
2024-12-05 15:24:19,548 [MainThread  ] [INFO ]  
CPPTRAJ: Trajectory 

0

### Visualization

In [36]:
def plot_and_view(den,view,labels,axis='z'):
    axis = f'#{axis.upper()}'
    fig, ax = plt.subplots(1, 1, figsize=(8, 4))
    cls = den.columns[1:]
    cls = den.columns[1:]
    for i,cl in enumerate(cls):
        if i%2==0:
            ax.plot(den[axis], den[cl], label=labels[i//2])
            cl0 = cl
        else:
            ax.fill_between(den[axis], den[cl0] - den[cl], den[cl0] + den[cl],label='SD',color='red')
    ax.set_title('Mass Density');ax.set_xlabel('Å')
    ax.set_ylabel('Density'); ax.legend()

    # Put plot in a widget and display both widgets in a row
    out = Output()
    with out: plt.show()
    box = HBox([out,view]); display(box)
    view._remote_call('setSize', target='Widget', args=['100%', '100%'])
    view.control.spin([0, 1, 0], -1.57) 

In [38]:
# Mask
view = nv.show_file(input_top_path)
den = pd.read_csv(f'out/density_mask.dat', sep='\s+')
plot_and_view(den,view,['Membrane','Protein'])
view.update_ball_and_stick(color='blue')
view.update_cartoon(color='orange')

<a id="leaflets"></a>
***
## Leafleats assignation

Lyphilic is faster that fatslim, it algorithm is simpler, assumes a plane membrane, it let you do the assignation of leafleats for every frame and use this information for analysis like flip-flop <br>
***
**Building Blocks** used:
 - [LPPAssignLeaflets](https://biobb-mem.readthedocs.io/en/latest/lipyphilic_biobb.html) from **from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets**
***

![DPPC](../html/DPPC.png "DPPC")

In [12]:
from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets import lpp_assign_leaflets

prop = {
    'lipid_sel': lipid_sel,
    'steps': steps
}
lpp_assign_leaflets(input_top_path=input_top_path,
                    input_traj_path=input_traj_path,
                    output_leaflets_path= 'leaflets_data.csv',
                    properties=prop)

/home/rchaves/miniforge3/envs/biobb_mem/lib/python3.10/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


2024-12-04 12:43:28,108 [MainThread  ] [INFO ]  Module: biobb_mem.lipyphilic_biobb.lpp_assign_leaflets Version: 5.0.2
2024-12-04 12:43:28,109 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_e34b02ff-7e8d-49fa-b6dc-f396bfbb0b82 directory successfully created
2024-12-04 12:43:28,111 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_e34b02ff-7e8d-49fa-b6dc-f396bfbb0b82
2024-12-04 12:43:28,701 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_e34b02ff-7e8d-49fa-b6dc-f396bfbb0b82
2024-12-04 12:43:28,961 [MainThread  ] [INFO ]  Removed: ['/home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_e34b02ff-7e8d-49fa-b6dc-f396bfbb0b82']
2024-12-04 12:43:28,962 [MainThread  ] [INFO ]  


0

### Visualization

In [13]:
from biobb_mem.lipyphilic_biobb.lpp_assign_leaflets import display_nglview
view = display_nglview(input_top_path, 'leaflets_data.csv')
view.control.spin([0, 1, 0], -1.57)
view

NGLWidget()

<a id="channel"></a>
***
## Channel

Chanel... <br>
***
**Building Blocks** used:
 - [MDAHole](https://biobb-mem.readthedocs.io/en/latest/mdanalysis_biobb.html#mdanalysis_biobb.mda_hole.MDAHole) from **from biobb_mem.mdanalysis_biobb.mda_hole**
***

In [14]:
from biobb_mem.mdanalysis_biobb.mda_hole import mda_hole

prop = {
    'select': 'protein',
    'steps': steps,
    'sample': 0.1
}
mda_hole(input_top_path=input_top_path,
         input_traj_path=input_traj_path,
         output_hole_path='hole.vmd',
         properties=prop)

2024-12-04 12:45:05,265 [MainThread  ] [INFO ]  Module: biobb_mem.mdanalysis_biobb.mda_hole Version: 5.0.2
2024-12-04 12:45:05,266 [MainThread  ] [INFO ]  /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_cded5ead-dc7c-47d6-8a8b-ee6b4202800a directory successfully created
2024-12-04 12:45:05,268 [MainThread  ] [INFO ]  Copy: A01M6.pdb to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_cded5ead-dc7c-47d6-8a8b-ee6b4202800a


/home/rchaves/miniforge3/envs/biobb_mem/lib/python3.10/site-packages/MDAnalysis/analysis/hole2/__init__.py:47: DeprecationWarning: Deprecated in version 2.8.0
MDAnalysis.analysis.hole2 is deprecated in favour of the MDAKit madahole2 (https://www.mdanalysis.org/mdahole2/) and will be removed in MDAnalysis version 3.0.0
  warnings.warn(wmsg, category=DeprecationWarning)


2024-12-04 12:45:05,845 [MainThread  ] [INFO ]  Copy: A01M6.xtc to /home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_cded5ead-dc7c-47d6-8a8b-ee6b4202800a


/home/rchaves/miniforge3/envs/biobb_mem/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/rchaves/miniforge3/envs/biobb_mem/lib/python3.10/site-packages/MDAnalysis/lib/util.py:575: DeprecationWarning: This method is deprecated as of MDAnalysis version 2.7.0 and will be removed in version 3.0.0. Please use shutil.which instead.
  warnings.warn(wmsg, DeprecationWarning)


2024-12-04 12:45:18,455 [MainThread  ] [INFO ]  Removed: ['/home/rchaves/repo/biobb_wf_mem/biobb_wf_mem/notebooks/sandbox_cded5ead-dc7c-47d6-8a8b-ee6b4202800a']
2024-12-04 12:45:18,455 [MainThread  ] [INFO ]  


0

In [ ]:
from biobb_mem.mdanalysis_biobb.mda_hole import display_hole
display_hole(input_top_path, opacity=0.9)

In [15]:
#%conda install -c conda-forge vmd -y # si se instala en el entorno conda da problemas de clobber y jupyter no arranca
#%conda install libsqlite --force-reinstall -y # vmd breaks it

In [16]:
# TODO: cargar más frame (lento pero se resolvería con H5MD) y mirar por que algunos caneles salen torcidos
vmd_script = f"""
mol addfile {input_traj_path} type xtc first 0 last 1 step {steps} waitfor all
animate delete beg 0 end 0
source hole.vmd
"""

# Join the contents and write to a new file
with open('vmd_script.vmd', 'w') as fl:
    fl.write(vmd_script)

!vmd -pdb {input_top_path} -e vmd_script.vmd

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 16 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 52GB (83%)
Info) No CUDA accelerator devices available.
Info) OpenGL renderer: Mesa Intel(R) UHD Graphics 630 (CML GT2)
Info)   Features: STENCIL MSAA(4) MDE CVA MTX NPOT PP PS GLSL(OVFS) 
Info)   Full GLSL rendering mode is available.
Info)   Textures: 2-D (16384x16384), 3-D (512x512x512), Multitexture (8)
Info) Detected 8 XInput devices, 1 usable device:
Info)   [8] xwayland-pointer-gestures:16, type: 242